In [1]:
import pyodbc
import numpy as np
import pandas as pd
import pandas.io.sql as psql
from datetime import datetime as dt, timedelta, date, time
import tkinter as tk
from tkinter import messagebox
from tqdm import tqdm


In [3]:
# Create a Tkinter root window
root = tk.Tk()
root.withdraw()

# Show a warning message box with an OK button
messagebox.showerror("Warning", "This is a warning message.")

# Close the Tkinter root window
root.destroy()

In [2]:
# Testing is NaT

print("Check for NaT? ", np.isnat(np.datetime64(None)))

# result = (np.datetime64('2022-07-26 15:11:51.970') - np.datetime64('2022-07-18 10:22:35.990')).astype('timedelta64[s]') / 3600.0

# print(float(result))

date1 = dt.strptime('2023-09-26 09:00:00.000', '%Y-%m-%d %H:%M:%S.%f')
date2 = dt.strptime('2023-09-21 13:08:40.797', '%Y-%m-%d %H:%M:%S.%f')

result = (date1 - date2).total_seconds() / 3600.0
print(float(result))

Check for NaT?  True
115.85533416666667


TIME DIFFERENCE

In [5]:
# date_1_str = "2023-03-20 09:17:27.947"
# date_2_str = "2023-04-26 15:10:08.833"
date_1_str = "2023-03-20 09:17:27.947"
date_2_str = "2023-03-21 15:10:08.833"

date_1 = dt.strptime(date_1_str, '%Y-%m-%d %H:%M:%S.%f')
date_2 = dt.strptime(date_2_str, '%Y-%m-%d %H:%M:%S.%f')
workday_start_T = time(7,0)
workday_end_T = time(23,59,59,999)

workday_start = dt.combine(date_2.date(), workday_start_T)
workday_end = dt.combine(date_1.date(), workday_end_T)

print("The time difference is:", ((workday_end - date_1) + (date_2 - workday_start)).total_seconds() / 3600)

time_difference = (dt.strptime(date_2_str, '%Y-%m-%d %H:%M:%S.%f') - dt.strptime(date_1_str, '%Y-%m-%d %H:%M:%S.%f')).total_seconds() / 3600
time_difference_days = dt.strptime(date_2_str, '%Y-%m-%d %H:%M:%S.%f').date() - dt.strptime(date_1_str, '%Y-%m-%d %H:%M:%S.%f').date()
print(time_difference)
print(time_difference_days)
print(timedelta(days=1) - timedelta(days=2))
print(time_difference_days == timedelta(days=37))
print((date_1 - date_2).days)
print(timedelta(days=2) * 4)

new_date = dt.combine(date_1.date() + timedelta(days=2), time(0,0))
print(new_date)

The time difference is: 22.87774638861111
29.87802388888889
1 day, 0:00:00
-1 day, 0:00:00
False
-2
8 days, 0:00:00
2023-03-22 00:00:00


In [2]:
def delta_working_hours(actual_start, actual_end, calendar=True):
    """
    Function gets the difference between two datetime objects
    Calendar flag calculates raw time delta between two datetime objects
    Standard flag (calendar=False) calculates time delta between two datetime objects excluding weekends
    :param actual_start: The actual start time
    :type actual_start: datetime.datetime
    :param actual_end: The actual end time
    :type actual_end: datetime.datetime
    :param calendar: Flag, if True calculate as calendar, if False, calculate as standard working hours
    :return: The time delta between actual_start and actual_end
    :rtype: float
    """
    seconds_between = (actual_end - actual_start).total_seconds()

    # Invalid negative value
    if seconds_between < 0:
        return 0
    elif calendar:
        return seconds_between / 3600
    else:  # Standard time calculation
        workdays_difference = actual_end.date() - actual_start.date()
        standard_start_T = time(0, 0)
        standard_end_T = time(23, 59, 59, 9999)
        weekend_days = 0

        # When the work operation occurs on the same day or up to 1 day difference, return the plain difference
        if workdays_difference < timedelta(days=2):
            return seconds_between / 3600

        # Else:
        # Assign a new start or end of actual start or actual end if date falls on the weekend
        if actual_start.weekday() == 5:
            actual_start = dt.combine(actual_start.date() + timedelta(days=2), standard_start_T)
        elif actual_start.weekday() == 6:
            actual_start = dt.combine(actual_start.date() + timedelta(days=1), standard_start_T)
        if actual_end.weekday() == 5:
            actual_end = dt.combine(actual_end.date() - timedelta(days=1), standard_end_T)
        elif actual_end.weekday() == 6:
            actual_end = dt.combine(actual_end.date() - timedelta(days=2), standard_end_T)

        # Find weekend days in between the start and end
        days_between = (actual_end - actual_start).days
        for i in range(1, days_between):
            temp_date = actual_start + timedelta(days=i)
            if temp_date.weekday() in {5, 6}:
                weekend_days += 1

        # Return the delta of two timestamps in working standard time (Monday 0:00 - Friday 23:59:59.9999)
        return ((actual_end - actual_start).total_seconds() / 3600) - (weekend_days * 24)

In [3]:
date_1_str = "2022-11-16 22:13:24.927"
date_2_str = "2023-12-21 09:00:00.000"
# date_1_str = "2023-03-20 09:17:27.947"
# date_2_str = "2023-03-21 15:10:08.833"

date_1 = dt.strptime(date_1_str, '%Y-%m-%d %H:%M:%S.%f')
date_2 = dt.strptime(date_2_str, '%Y-%m-%d %H:%M:%S.%f')

print(delta_working_hours(date_1, date_2, calendar=False))


6850.776409166667


STANDARDIZED TIME

In [2]:
def delta_working_hours_std(actual_start, actual_end):
    workdays_difference = actual_end.date() - actual_start.date()
    days_between = (actual_end - actual_start).days
    standard_start_T = time(0, 0)
    standard_end_T = time(23, 59, 59, 9999)
    weekend_days = 0

    # Invalid negative value
    if days_between < 0:
        return None

    # When the work operation occurs on the same day or up to 1 day difference, return the plain difference
    if workdays_difference < timedelta(days=2):
        return (actual_end - actual_start).total_seconds() / 3600

    # Else:
    # Assign a new start or end of actual start or actual end fall in the weekend
    if actual_start.weekday() == 5:
        actual_start = dt.combine(actual_start.date() + timedelta(days=2), standard_start_T)
    elif actual_start.weekday() == 6:
        actual_start = dt.combine(actual_start.date() + timedelta(days=1), standard_start_T)
    if actual_end.weekday() == 5:
        actual_end = dt.combine(actual_end.date() - timedelta(days=1), standard_end_T)
    elif actual_end.weekday() == 6:
        actual_end = dt.combine(actual_end.date() - timedelta(days=2), standard_end_T)

    # Find weekend days in between the start and end
    for i in range(1, days_between):
        temp_date = actual_start + timedelta(days=i)
        if temp_date.weekday() in {5, 6}:
            weekend_days += 1

    # Return the delta of two timestamps in working standard time (Monday 0:00 - Friday 23:59:59.9999)
    return ((actual_end - actual_start).total_seconds() / 3600) - (weekend_days * 24)


def delta_working_hours(actual_start, actual_end, calendar=False):
    seconds_between = (actual_end - actual_start).total_seconds()

    # Invalid negative value
    if seconds_between < 0:
        return None
    elif calendar:
        return seconds_between / 3600
    else:  # Standard time calculation
        workdays_difference = actual_end.date() - actual_start.date()
        standard_start_T = time(0, 0)
        standard_end_T = time(23, 59, 59, 9999)
        weekend_days = 0

        # When the work operation occurs on the same day or up to 1 day difference, return the plain difference
        if workdays_difference < timedelta(days=2):
            return seconds_between / 3600

        # Else:
        # Assign a new start or end of actual start or actual end if date falls on the weekend
        if actual_start.weekday() == 5:
            actual_start = dt.combine(actual_start.date() + timedelta(days=2), standard_start_T)
        elif actual_start.weekday() == 6:
            actual_start = dt.combine(actual_start.date() + timedelta(days=1), standard_start_T)
        if actual_end.weekday() == 5:
            actual_end = dt.combine(actual_end.date() - timedelta(days=1), standard_end_T)
        elif actual_end.weekday() == 6:
            actual_end = dt.combine(actual_end.date() - timedelta(days=2), standard_end_T)

        # Find weekend days in between the start and end
        days_between = (actual_end - actual_start).days
        for i in range(1, days_between):
            temp_date = actual_start + timedelta(days=i)
            if temp_date.weekday() in {5, 6}:
                weekend_days += 1

        # Return the delta of two timestamps in working standard time (Monday 0:00 - Friday 23:59:59.9999)
        return ((actual_end - actual_start).total_seconds() / 3600) - (weekend_days * 24)


In [3]:
i = 0
date_1_str = "2022-11-23 14:34:50.363"
date_2_str = "2022-11-23 22:03:23.923"
# date_1_str = "2023-03-20 09:17:27.947"
# date_2_str = "2023-03-21 15:10:08.833"

date_1 = dt.strptime(date_1_str, '%Y-%m-%d %H:%M:%S.%f')
date_2 = dt.strptime(date_2_str, '%Y-%m-%d %H:%M:%S.%f')

standard_delta = delta_working_hours(date_1, date_2, i == 1)


print("std:", standard_delta)

print("calendar:", (date_2 - date_1).total_seconds() / 3600)


std: 7.47598888888889
calendar: 7.47598888888889


In [10]:


my_set = set()

my_list = []
my_list.append((1, 2))
my_list.append((3, 4))
my_set.update(my_list)
my_set

print(dt.now() - timedelta(days=262))

2022-09-17 14:33:41.510801
